In [6]:
pip install pulp

     ---------------------------------------- 14.3/14.3 MB 9.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
#problem two
import pulp

# Create a linear programming problem
lp_problem = pulp.LpProblem("LP Problem", pulp.LpMaximize)

# Define the variables x1 to x10
x = [pulp.LpVariable(f'x{i}', 0, 1, pulp.LpBinary) for i in range(1, 11)]

# Define additional binary variables y and z
y = pulp.LpVariable("y", 0, 1, pulp.LpBinary)
z = pulp.LpVariable("z", 0, 1, pulp.LpBinary)

# Define the objective function to maximize
lp_problem += 4*x[0] + 5*x[1] + 3*x[2] + 4.3*x[3] + x[4] + 1.5*x[5] + 2.5*x[6] + 0.3*x[7] + x[8] + 2*x[9]

# Define the constraints
lp_problem += 2*x[0] + 3*x[1] + 1.5*x[2] + 2.2*x[3] + 0.5*x[4] + 1.5*x[5] + 2.5*x[6] + 0.1*x[7] + 0.6*x[8] + x[9] <= 5
lp_problem += x[0] + x[1] + x[2] + x[3] <= 1
lp_problem += x[4] + x[5] + x[6] <= 1
lp_problem += x[7] + x[8] + x[9] <= 1

# Additional constraints to represent (x2 + x4) * (1 - (x6 + x7)) == 0
"""

#lp_problem += x[1] + x[3] <= 1  # x2 + x4 <= 1
#lp_problem += x[6] + x[7] <= 1  # x6 + x7 <= 1
lp_problem += (x[1] + x[3])*(1-(x[6]+x[7])) < 0
lp_problem += (x[1] + x[3])*(1-(x[6]+x[7])) > 0
"""
"""
lp_problem += y >= x[1] + x[3] - 1
lp_problem += y <= x[1] + x[3]
lp_problem += z >= 1 - (x[6] + x[7])
lp_problem += z <= 1 - x[6] - x[7]
lp_problem += y + z == 1"""
lp_problem += x[1] <= x[5] + x[6]
lp_problem += x[3] <= x[5] + x[6]
# Solve the problem
lp_problem.solve()

# Print the results
print("Status:", pulp.LpStatus[lp_problem.status])
print("Optimal Solution:")
for var in lp_problem.variables():
    print(f"{var.name} = {var.varValue}")
print("Optimal Value =", pulp.value(lp_problem.objective))



Status: Optimal
Optimal Solution:
x1 = 0.0
x10 = 1.0
x2 = 0.0
x3 = 0.0
x4 = 1.0
x5 = 0.0
x6 = 1.0
x7 = 0.0
x8 = 0.0
x9 = 0.0
Optimal Value = 7.8


In [7]:
#problem 3
# Use %pip install pulp
import pulp

# Cash flows for the first two years
cash_flow_year1 = [-90000, -80000, -50000, -20000, -40000,-80000,-80000]
cash_flow_year2 = [-58000, -80000, -100000, -64000, -50000,-20000,-100000]

# Net present values
npv = [150000, 200000, 100000, 100000, 120000,150000,240000]

# Budget and interest rate
budget_year1 = 250000
budget_year2 = 250000
interest_rate = 0.1

# Create the LP problem
problem = pulp.LpProblem("Project_Selection", pulp.LpMaximize)

# Decision variables
x = pulp.LpVariable.dicts("x", range(7), 0, 1, cat="Integer")

# Objective function
problem += pulp.lpSum([x[i] * npv[i] for i in range(7)]), "Maximize Total NPV"

# Constraints
problem += pulp.lpSum([x[i] * -cash_flow_year1[i] for i in range(7)]) <= budget_year1, "Year 1 Budget"
problem += pulp.lpSum([x[i] * -cash_flow_year2[i] for i in range(7)]) <= budget_year2 + interest_rate * (budget_year1 - pulp.lpSum([x[i] * cash_flow_year1[i] for i in range(5)])), "Year 2 Budget"

# Solve the problem
problem.solve()

# Print the results
print("Selected Projects:")
for i in range(7):
    if x[i].value() == 1:
        print(f"Project {i + 1}")



Selected Projects:
Project 4
Project 5
Project 6
Project 7


In [17]:
#problem 8
# Calculation for part B

g = 400
d = 1 / 1.1
K = [213.81, 211.45, 208.17, 203.58, 197.13, 187.96, 174.79, 155.47, 126.28, 80.00]
x=[0 for i in range(11)]
z=[0 for i in range(10)]

x[0] = 50000
# Calculate z_j and x_{j+1} for each year.
for j in range(9):
    z[j] = ((g - d * K[j+1]) * x[j]) / 1000
    x[j+1] = x[j] - z[j]
z[9]=g*x[9]/1000
x[10] = x[9] - z[9]

# Calculate remaining gold after 10 years
remaining_gold = x[-1]

# Calculate the value of the mine after 10 years
K_10 = 220  # K-value for an infinite time horizon


remaining_gold

2393.0575899038395

In [14]:
#problem 9
initial_reserves = 100000
oil_price = 10
interest_rate = 0.10
years = 3

actions = [("No Pumping", 0, 0), 
           ("Normal Pumping", 0.20, 50000), 
           ("Enhanced Pumping", 0.36, 120000)]

# Dynamic programming table: rows = years, columns = reserves
dp = [[0] * (initial_reserves + 1) for _ in range(years+1)]
actions_taken = [[None] * (initial_reserves + 1) for _ in range(years+1)]

for year in reversed(range(years)):
    for reserves in range(initial_reserves+1):
        best_profit = -float('inf')
        best_action = None
        
        for action_name, depletion_rate, cost in actions:
            barrels_pumped = int(reserves * depletion_rate)
            revenue_from_oil = barrels_pumped * oil_price
            remaining_reserves = reserves - barrels_pumped
            
            profit_this_year = revenue_from_oil - cost
            total_profit = profit_this_year + dp[year+1][remaining_reserves] / (1 + interest_rate)
            
            if total_profit > best_profit:
                best_profit = total_profit
                best_action = action_name
        
        dp[year][reserves] = best_profit
        actions_taken[year][reserves] = best_action

max_profit = dp[0][initial_reserves]
strategy = []
res = initial_reserves
for year in range(years):
    strategy.append(actions_taken[year][res])
    if actions_taken[year][res] == "No Pumping":
        pass
    elif actions_taken[year][res] == "Normal Pumping":
        res -= int(res * 0.20)
    else:
        res -= int(res * 0.36)

print(f"Maximum present value of profits: ${max_profit:,.2f}")
print(f"Optimal pumping strategy: {', '.join(strategy)}")


Maximum present value of profits: $366,743.80
Optimal pumping strategy: Enhanced Pumping, Enhanced Pumping, Normal Pumping


In [16]:
#problem 12
D1 = 10000000
G = 0.10
g = 0.05
r = 0.15
k = 5

PV_high = sum([D1 * (1 + G)**t / (1 + r)**(t+1) for t in range(k)])
PV_stable0 = D1 * (1 + G)**k / ((r - g) * (1 + r)**k)

V = PV_high + PV_stable0
V

119929189.60347521